In [161]:
import pandas as pd

df=pd.read_csv("C:\\Users\\SAMEER MUJAWR\\Downloads\\DBDA\\Project\\Recommendation System\\Zomato Recommandation System\\zomato.csv")
df.shape

(51717, 17)

In [162]:
df = df.drop(["reviews_list"], axis=1)

In [163]:
df = df.map(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)
df = df.replace('\r', ' ', regex=True)
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555 +91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447 +91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,[],Buffet,Banashankari


In [164]:
df.to_csv("C:\\Users\\SAMEER MUJAWR\\Downloads\\DBDA\\Project\\Recommendation System\\Zomato Recommandation System\\zomato_new.csv",index=False)

In [165]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Recommendation System").config("spark.driver.memory", "8g").config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "8") \
    .getOrCreate()


In [166]:
from pyspark.sql.functions import col, regexp_replace,regexp_extract
from pyspark.sql.functions import col, avg, when, coalesce,split,count
df=spark.read.csv("C:/Users/SAMEER MUJAWR/Downloads/DBDA/Project/Recommendation System/Zomato Recommandation System/zomato_new.csv",header=True,inferSchema=True)
null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
null_counts.show()

+---+-------+----+------------+----------+----+-----+-----+--------+---------+----------+--------+---------------------------+---------+---------------+---------------+
|url|address|name|online_order|book_table|rate|votes|phone|location|rest_type|dish_liked|cuisines|approx_cost(for two people)|menu_item|listed_in(type)|listed_in(city)|
+---+-------+----+------------+----------+----+-----+-----+--------+---------+----------+--------+---------------------------+---------+---------------+---------------+
|  0|      0|   0|           0|         0|7775|    0| 1208|      21|      227|     28078|      45|                        346|        0|              0|              0|
+---+-------+----+------------+----------+----+-----+-----+--------+---------+----------+--------+---------------------------+---------+---------------+---------------+



In [167]:
df = df.drop("url","phone",'address',"rest_type","dish_liked","listed_in(city)", 'menu_item', 'votes')
df.show()

+--------------------+------------+----------+-----+------------+--------------------+---------------------------+---------------+
|                name|online_order|book_table| rate|    location|            cuisines|approx_cost(for two people)|listed_in(type)|
+--------------------+------------+----------+-----+------------+--------------------+---------------------------+---------------+
|               Jalsa|         Yes|       Yes|4.1/5|Banashankari|North Indian, Mug...|                        800|         Buffet|
|      Spice Elephant|         Yes|        No|4.1/5|Banashankari|Chinese, North In...|                        800|         Buffet|
|     San Churro Cafe|         Yes|        No|3.8/5|Banashankari|Cafe, Mexican, It...|                        800|         Buffet|
|Addhuri Udupi Bho...|          No|        No|3.7/5|Banashankari|South Indian, Nor...|                        300|         Buffet|
|       Grand Village|          No|        No|3.8/5|Basavanagudi|North Indian, Raj.

In [168]:
df=df.withColumn("cuisines", split(df["cuisines"], ","))
df.select("cuisines").distinct().show()

+--------------------+
|            cuisines|
+--------------------+
|[Andhra,  South I...|
|[South Indian,  N...|
|[Lebanese,  Medit...|
|[Bengali,  North ...|
|[North Indian,  B...|
|[South Indian,  C...|
|[Desserts,  Bever...|
|[Chinese,  South ...|
|[Chinese,  South ...|
|[Fast Food,  Arab...|
|[Cafe,  Italian, ...|
|[North Indian,  C...|
|[American,  Sandw...|
|[Rajasthani,  Nor...|
|[Continental,  Ch...|
|[Chinese,  Seafoo...|
|[Arabian,  Middle...|
|[Chinese,  Fast F...|
|[Seafood,  Biryan...|
|[Mughlai,  North ...|
+--------------------+
only showing top 20 rows



In [169]:
df = df.withColumn("rate", regexp_extract(col("rate"), r"(\d+\.?\d*)", 0).cast("float"))
# df = df.filter((col("rate") != "NEW") & (col("rate") != "-"))
df.select("rate").distinct().show()

+----+
|rate|
+----+
| 2.5|
| 3.8|
| 2.2|
| 4.6|
| 3.4|
| 4.2|
| 3.2|
| 3.6|
| 3.0|
| 2.9|
|NULL|
| 2.6|
| 3.3|
| 2.8|
| 4.4|
| 3.5|
| 4.1|
| 4.8|
| 2.4|
| 2.7|
+----+
only showing top 20 rows



In [170]:
df = (df
          .withColumnRenamed("approx_cost(for two people)", "cost")
          .withColumnRenamed("listed_in(type)", "type"))
df.show()


+--------------------+------------+----------+----+------------+--------------------+----+------+
|                name|online_order|book_table|rate|    location|            cuisines|cost|  type|
+--------------------+------------+----------+----+------------+--------------------+----+------+
|               Jalsa|         Yes|       Yes| 4.1|Banashankari|[North Indian,  M...| 800|Buffet|
|      Spice Elephant|         Yes|        No| 4.1|Banashankari|[Chinese,  North ...| 800|Buffet|
|     San Churro Cafe|         Yes|        No| 3.8|Banashankari|[Cafe,  Mexican, ...| 800|Buffet|
|Addhuri Udupi Bho...|          No|        No| 3.7|Banashankari|[South Indian,  N...| 300|Buffet|
|       Grand Village|          No|        No| 3.8|Basavanagudi|[North Indian,  R...| 600|Buffet|
|     Timepass Dinner|         Yes|        No| 3.8|Basavanagudi|      [North Indian]| 600|Buffet|
|Rosewood Internat...|          No|        No| 3.6| Mysore Road|[North Indian,  S...| 800|Buffet|
|              Onest

In [171]:
df = df.withColumn("cost", round(col("cost").cast("float"),2))
df.select("cost").distinct().show()

+-----+
| cost|
+-----+
|550.0|
|500.0|
|190.0|
|350.0|
|700.0|
|650.0|
|750.0|
|850.0|
|180.0|
|100.0|
| NULL|
| 40.0|
|250.0|
|400.0|
|200.0|
| 50.0|
| 80.0|
|150.0|
|450.0|
|800.0|
+-----+
only showing top 20 rows



In [172]:
avg_cost_per_restaurant = df.groupBy("location").agg(avg("cost").alias("Avg Cost"))

df = df.join(avg_cost_per_restaurant, on="location", how="left")

df= df.withColumn("cost", coalesce(col("cost"), col("Avg Cost")))

df = df.drop("Avg Cost")

df.show()


+------------+--------------------+------------+----------+----+--------------------+-----+------+
|    location|                name|online_order|book_table|rate|            cuisines| cost|  type|
+------------+--------------------+------------+----------+----+--------------------+-----+------+
|Banashankari|               Jalsa|         Yes|       Yes| 4.1|[North Indian,  M...|800.0|Buffet|
|Banashankari|      Spice Elephant|         Yes|        No| 4.1|[Chinese,  North ...|800.0|Buffet|
|Banashankari|     San Churro Cafe|         Yes|        No| 3.8|[Cafe,  Mexican, ...|800.0|Buffet|
|Banashankari|Addhuri Udupi Bho...|          No|        No| 3.7|[South Indian,  N...|300.0|Buffet|
|Basavanagudi|       Grand Village|          No|        No| 3.8|[North Indian,  R...|600.0|Buffet|
|Basavanagudi|     Timepass Dinner|         Yes|        No| 3.8|      [North Indian]|600.0|Buffet|
| Mysore Road|Rosewood Internat...|          No|        No| 3.6|[North Indian,  S...|800.0|Buffet|
|Banashank

In [173]:
avg_rating_per_location = df.groupBy("location").agg(avg("rate").alias("Location Avg Rating"))

df = df.join(avg_rating_per_location, on="location", how="left")

df = df.withColumn("rate", coalesce(col("rate"), col("Location Avg Rating")))

df = df.drop("Location Avg Rating")

df = df.withColumn("rate", round(col("rate"), 2))

df.show()


+------------+--------------------+------------+----------+----+--------------------+-----+------+
|    location|                name|online_order|book_table|rate|            cuisines| cost|  type|
+------------+--------------------+------------+----------+----+--------------------+-----+------+
|Banashankari|               Jalsa|         Yes|       Yes| 4.1|[North Indian,  M...|800.0|Buffet|
|Banashankari|      Spice Elephant|         Yes|        No| 4.1|[Chinese,  North ...|800.0|Buffet|
|Banashankari|     San Churro Cafe|         Yes|        No| 3.8|[Cafe,  Mexican, ...|800.0|Buffet|
|Banashankari|Addhuri Udupi Bho...|          No|        No| 3.7|[South Indian,  N...|300.0|Buffet|
|Basavanagudi|       Grand Village|          No|        No| 3.8|[North Indian,  R...|600.0|Buffet|
|Basavanagudi|     Timepass Dinner|         Yes|        No| 3.8|      [North Indian]|600.0|Buffet|
| Mysore Road|Rosewood Internat...|          No|        No| 3.6|[North Indian,  S...|800.0|Buffet|
|Banashank

In [174]:
null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

null_counts.show()

+--------+----+------------+----------+----+--------+----+----+
|location|name|online_order|book_table|rate|cuisines|cost|type|
+--------+----+------------+----------+----+--------+----+----+
|      21|   0|           0|         0|  29|      45|  26|   0|
+--------+----+------------+----------+----+--------+----+----+



In [175]:
df = df.na.drop(subset=["cost","cuisines","rate"])

df.show()
df = df.dropDuplicates()

+------------+--------------------+------------+----------+----+--------------------+-----+------+
|    location|                name|online_order|book_table|rate|            cuisines| cost|  type|
+------------+--------------------+------------+----------+----+--------------------+-----+------+
|Banashankari|               Jalsa|         Yes|       Yes| 4.1|[North Indian,  M...|800.0|Buffet|
|Banashankari|      Spice Elephant|         Yes|        No| 4.1|[Chinese,  North ...|800.0|Buffet|
|Banashankari|     San Churro Cafe|         Yes|        No| 3.8|[Cafe,  Mexican, ...|800.0|Buffet|
|Banashankari|Addhuri Udupi Bho...|          No|        No| 3.7|[South Indian,  N...|300.0|Buffet|
|Basavanagudi|       Grand Village|          No|        No| 3.8|[North Indian,  R...|600.0|Buffet|
|Basavanagudi|     Timepass Dinner|         Yes|        No| 3.8|      [North Indian]|600.0|Buffet|
| Mysore Road|Rosewood Internat...|          No|        No| 3.6|[North Indian,  S...|800.0|Buffet|
|Banashank

In [ ]:
null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

null_counts.show()


+--------+----+------------+----------+----+--------+----+----+
|location|name|online_order|book_table|rate|cuisines|cost|type|
+--------+----+------------+----------+----+--------+----+----+
|       0|   0|           0|         0|   0|       0|   0|   0|
+--------+----+------------+----------+----+--------+----+----+



In [177]:
# Convert the PySpark DataFrame to a Pandas DataFrame
pandas_df = df.select("location").toPandas()

# Get unique values from the 'location' column and convert to a Python list
unique_locations = pandas_df["location"].dropna().unique().tolist()

# Print the unique locations
print(unique_locations)


['Kumaraswamy Layout', 'Jayanagar', 'Basavanagudi', 'Banashankari', 'Mysore Road', 'BTM', 'Bannerghatta Road', 'Marathahalli', 'HSR', 'Sarjapur Road', 'Bellandur', 'Brigade Road', 'JP Nagar', 'City Market', 'Vasanth Nagar', 'Richmond Road', 'MG Road', 'Indiranagar', 'Residency Road', 'Shanti Nagar', 'Ulsoor', 'Wilson Garden', 'Race Course Road', 'South Bangalore', 'Koramangala 5th Block', 'Domlur', 'Shivajinagar', 'Kanakapura Road', 'Frazer Town', 'Cunningham Road', 'Lavelle Road', 'Koramangala 7th Block', 'Church Street', 'Koramangala 8th Block', 'St. Marks Road', 'Infantry Road', 'Uttarahalli', 'Rajarajeshwari Nagar', 'Commercial Street', 'Jalahalli', 'Whitefield', 'Bommanahalli', 'Koramangala 6th Block', 'Brookefield', 'ITPL Main Road, Whitefield', 'Koramangala 2nd Block', 'Varthur Main Road, Whitefield', 'Majestic', 'Hosur Road', 'Koramangala 1st Block', 'Koramangala', 'Koramangala 4th Block', 'Koramangala 3rd Block', 'Sanjay Nagar', 'Ejipura', 'Electronic City', 'Kammanahalli', 'B

In [179]:
from pyspark.sql.functions import col, explode

# Convert the Spark DataFrame to a Pandas DataFrame, flattening the array column
pandas_df = df.select(explode(col("cuisines")).alias("cuisine")).toPandas()

# Get unique values from the flattened column and convert to a Python list
unique_cuisines = pandas_df["cuisine"].dropna().unique().tolist()

# Print the result
print(unique_cuisines)
print(len(unique_cuisines))
print(len(unique_locations))


['Goan', ' Seafood', ' North Indian', ' Chinese', ' Biryani', 'South Indian', 'Juices', 'Sandwich', ' Beverages', ' Street Food', 'North Indian', ' South Indian', ' Fast Food', 'Chinese', 'Desserts', ' Momos', 'Pizza', ' Italian', ' Thai', ' BBQ', 'Steak', ' Continental', ' Mughlai', ' Arabian', ' Kerala', 'Fast Food', 'American', ' Tex-Mex', ' Burger', 'Biryani', 'Mangalorean', 'Continental', ' Mexican', ' Steak', ' Bakery', ' Finger Food', 'Andhra', 'Bakery', ' Cafe', ' Coffee', ' Desserts', ' Sandwich', 'Italian', ' Rolls', 'Burger', ' American', ' Kebab', 'Street Food', ' French', 'Cafe', 'Seafood', 'Finger Food', 'Healthy Food', ' Juices', ' Tea', ' Mediterranean', ' Iranian', ' Andhra', 'Mithai', ' Pizza', 'Salad', 'Kerala', 'Beverages', 'Arabian', ' Ice Cream', 'Ice Cream', 'Tea', 'Thai', ' Vietnamese', 'Hyderabadi', 'BBQ', 'Mughlai', ' Salad', 'Bengali', ' Bihari', ' Lucknowi', 'Mexican', ' Tibetan', ' Afghani', 'Australian', ' Healthy Food', 'Mediterranean', ' European', ' Mit

In [ ]:
num_rows = df.count()

num_columns = len(df.columns)

print(f"Rows: {num_rows}, Columns: {num_columns}")


Rows: 24850, Columns: 8


In [ ]:
# from distutils.version import LooseVersion
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer

location_indexer = StringIndexer(inputCol="location", outputCol="location_id")
name_indexer = StringIndexer(inputCol="name", outputCol="name_id")

df = location_indexer.fit(df).transform(df)
df = name_indexer.fit(df).transform(df)


als = ALS(
    maxIter=10,
    regParam=0.1,
    userCol="location_id",
    itemCol="name_id",
    ratingCol="rate", 
    coldStartStrategy="drop"
)

model = als.fit(df)


In [ ]:
from pyspark.sql.functions import col, explode

location_recommendations = model.recommendForAllUsers(5)

exploded_recommendations = location_recommendations.withColumn("recommendation", explode(col("recommendations")))

final_recommendations = exploded_recommendations.select(
    col("location_id"),
    col("recommendation.name_id").alias("name_id"),
    col("recommendation.rating").alias("predicted_rating")
)

result = final_recommendations.join(df, on=["location_id", "name_id"], how="inner") \
                             .select("location", "name", "predicted_rating") \
                             .distinct()
result.orderBy("location", "predicted_rating", ascending=False).show(truncate=False)


+---------------+----------------------------------------+----------------+
|location       |name                                    |predicted_rating|
+---------------+----------------------------------------+----------------+
|Yelahanka      |Cheta's Kitchen                         |3.825882        |
|Yelahanka      |Red Chillies Curries Point              |3.5708234       |
|Yelahanka      |Prashanth Naati Corner                  |3.4334838       |
|Yelahanka      |Twist N Roll                            |3.3353841       |
|Whitefield     |Punjab Grill                            |4.7580833       |
|Whitefield     |Flechazo                                |4.739569        |
|Whitefield     |AB's - Absolute Barbecues               |4.6934066       |
|Whitefield     |You Mee                                 |4.589632        |
|Vasanth Nagar  |Belgian Waffle Factory                  |4.750819        |
|Ulsoor         |Republic Of Noodles - Lemon Tree Premier|4.403236        |
|Ulsoor     

In [ ]:
input_location = "Banashankari"

filtered_recommendations = df.filter(col("location") == input_location) \
    .select("name", "rate", "cuisines", "cost") \
    .orderBy(col("rate").desc())
filtered_recommendations.show(5, truncate=False)


+-------------+----+------------------------+-----+
|name         |rate|cuisines                |cost |
+-------------+----+------------------------+-----+
|Taaza Thindi |4.7 |[South Indian]          |100.0|
|Onesta       |4.6 |[Pizza,  Cafe,  Italian]|600.0|
|Onesta       |4.6 |[Pizza,  Cafe,  Italian]|600.0|
|Onesta       |4.6 |[Pizza,  Cafe,  Italian]|600.0|
|Poonam Sweets|4.4 |[Mithai]                |150.0|
+-------------+----+------------------------+-----+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import array_contains

input_cuisine = "North Indian"

filtered_recommendations = df.filter(array_contains(col("cuisines"), input_cuisine)) \
    .select("name", "rate", "location", "cost") \
    .orderBy(col("rate").desc())
filtered_recommendations.show(5, truncate=False)


+---------------+----+------------+------------------+
|name           |rate|location    |cost              |
+---------------+----+------------+------------------+
|Punjab Grill   |4.9 |Whitefield  |434.05749032614705|
|Punjab Grill   |4.9 |Malleshwaram|354.86531986531986|
|Punjab Grill   |4.9 |Malleshwaram|354.86531986531986|
|Barbeque Nation|4.8 |JP Nagar    |429.76506639427987|
|The Black Pearl|4.8 |Marathahalli|419.59418534221686|
+---------------+----+------------+------------------+
only showing top 5 rows



In [ ]:
df.show(5)

+------------------+--------------------+------------+----------+----+--------------------+-----+--------+-----------+-------+
|          location|                name|online_order|book_table|rate|            cuisines| cost|    type|location_id|name_id|
+------------------+--------------------+------------+----------+----+--------------------+-----+--------+-----------+-------+
|Kumaraswamy Layout|            Goa 0 Km|         Yes|       Yes| 3.6|[Goan,  Seafood, ...|800.0|Delivery|       43.0| 6946.0|
|         Jayanagar|Namma Brahmin's Idli|         Yes|        No| 3.6|      [South Indian]|100.0|Delivery|        9.0| 4725.0|
|      Basavanagudi|           Cane-O-La|         Yes|        No| 3.8|            [Juices]|100.0|Delivery|       18.0| 6560.0|
|      Banashankari|  Sai Super Sandwich|         Yes|        No|3.65|[Sandwich,  Bever...|200.0|Desserts|       13.0| 2476.0|
|      Banashankari|         Udupi Ruchi|         Yes|        No| 3.9|[South Indian,  N...|200.0|Dine-out|     

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, concat_ws
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression

df = df.withColumn("cuisines_str", concat_ws(",", col("cuisines"))).drop("cuisines")

categorical_columns = ["name", "online_order", "book_table", "cuisines_str", "type", "location"]

indexers = [StringIndexer(inputCol=col, outputCol=col + "_index") for col in categorical_columns]
encoders = [OneHotEncoder(inputCol=col + "_index", outputCol=col + "_vec") for col in categorical_columns]

feature_columns = [col + "_vec" for col in categorical_columns] + [col for col in df.columns if col not in categorical_columns + ["cost"]]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

pipeline = Pipeline(stages=indexers + encoders + [assembler])

processed_data = pipeline.fit(df).transform(df)

final_data = processed_data.select("features", col("cost").alias("label"))

train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=42)

lr = LinearRegression(featuresCol="features", labelCol="label")
model = lr.fit(train_data)

predictions = model.transform(test_data)

evaluator_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mae")
evaluator_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")

rmse = evaluator_rmse.evaluate(predictions)
print(f"RMSE (Root Mean Squared Error): {rmse}")

mae = evaluator_mae.evaluate(predictions)
print(f"MAE (Mean Absolute Error): {mae}")

r2 = evaluator_r2.evaluate(predictions)
print(f"R² (R-Squared): {r2}")
predictions.select("label", "prediction").show()


RMSE (Root Mean Squared Error): 82.46556657707407
MAE (Mean Absolute Error): 37.9103018483598
R² (R-Squared): 0.792857984073061
+-----+------------------+
|label|        prediction|
+-----+------------------+
|650.0| 716.7676915185725|
|900.0| 876.4686256873098|
|900.0| 879.4595367570539|
|400.0| 383.4344299411883|
|250.0|219.71118509365186|
|300.0|247.74801859830143|
|400.0|507.34090736970415|
|600.0|  615.995400806944|
|750.0| 738.0618334994151|
|750.0| 744.0422368322457|
|750.0| 749.0591284188696|
|750.0| 741.7705378421074|
|750.0|  767.245555506571|
|500.0| 494.0198755922897|
|500.0| 383.3057619043059|
|400.0|415.73740038233785|
|500.0|501.24626079846104|
|400.0|398.45629984210103|
|750.0| 743.9556520398469|
|600.0| 602.9947059515791|
+-----+------------------+
only showing top 20 rows



In [ ]:
df_pandas = df.toPandas()
df_pandas.to_csv("C:/Users/SAMEER MUJAWR/Downloads/DBDA/Project/Recommendation System/Zomato Recommandation System/final_output.csv", index=False)


In [ ]:
# new_data = spark.createDataFrame([
#     ("Banashankari", "New Cafe", "Yes", "No", 4.2, 300.0, "Cafe,Italian", "Delivery", 13, 7859),
#     ("Basavanagudi", "Another Restaurant", "No", "Yes", 3.8, 500.0, "North Indian", "Dine-out", 18, 3489)
# ], ["location", "name", "online_order", "book_table", "rate", "cost", "cuisines_str", "type", "location_id", "name_id"])



# # processed_new_data = pipeline.fit(df).transform(new_data)

# # predictions_new = model.transform(processed_new_data)

# # predictions_new.select("name", "location", "prediction").show()


# processed_new_data = pipeline.fit(df).transform(df.loc[:2,:])

# predictions_new = model.transform(processed_new_data)

# predictions_new.select("name", "location", "prediction").show()


In [ ]:
# print("Schema of processed_new_data:")
# processed_new_data.printSchema()

# print("Schema of predictions_new:")
# predictions_new.printSchema()
